In [19]:
import os, subprocess
import json
import uproot
import awkward as ak
import numpy as np
from coffea import processor, util, hist

%matplotlib inline
from matplotlib import lines as mlines
import matplotlib.pyplot as plt
from cycler import cycler

In [20]:
lumis = {}
lumis['2016'] = 35.9
lumis['2017'] = 41.5
lumis['2018'] = 59.9

nfiles_mc = {}
nfiles_mc['2016'] = 64
nfiles_mc['2017'] = 89
nfiles_mc['2018'] = 106

nfiles_data = {}
nfiles_data['2016'] = 16
nfiles_data['2017'] = 19
nfiles_data['2018'] = 27

with open('xsec.json') as f:
  xs = json.load(f)

with open('pmap.json') as f:
  pmap = json.load(f)

In [21]:
year = '2018'
outsum = processor.dict_accumulator()

In [22]:
# Load all MC
for n in range(1,nfiles_mc[year]+1):
    print(n)
    filename = 'condor/outfiles/'+year+'_'+str(n)+'.coffea'
    if os.path.isfile(filename):
        out = util.load(filename)
        outsum.add(out)
    else:
        print("File " + filename + " is missing")
        
scale_lumi = {k: xs[k] * 1000 *lumis[year] / w for k, w in outsum['sumw'].items()}
outsum['templates1'].scale(scale_lumi, 'dataset')

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89


In [23]:
# Load all data
for n in range(1,nfiles_data[year]+1):
    print(n)
    filename = 'condor/outdata/'+year+'_'+str(n)+'.coffea'
    if os.path.isfile(filename):
        out = util.load(filename)
        outsum.add(out)
    else:
        print("File " + filename + " is missing")
        
# BE CAREFUL NOT TO UNBLIND
# Can you figure out how to blind it at this step???

1
File condor/outdata/2017_1.coffea is missing
2
File condor/outdata/2017_2.coffea is missing
3
File condor/outdata/2017_3.coffea is missing
4
File condor/outdata/2017_4.coffea is missing
5
File condor/outdata/2017_5.coffea is missing
6
File condor/outdata/2017_6.coffea is missing
7
File condor/outdata/2017_7.coffea is missing
8
File condor/outdata/2017_8.coffea is missing
9
File condor/outdata/2017_9.coffea is missing
10
File condor/outdata/2017_10.coffea is missing
11
File condor/outdata/2017_11.coffea is missing
12
File condor/outdata/2017_12.coffea is missing
13
File condor/outdata/2017_13.coffea is missing
14
File condor/outdata/2017_14.coffea is missing
15
File condor/outdata/2017_15.coffea is missing
16
File condor/outdata/2017_16.coffea is missing
17
File condor/outdata/2017_17.coffea is missing
18
File condor/outdata/2017_18.coffea is missing
19
File condor/outdata/2017_19.coffea is missing


In [24]:
templates = outsum['templates1'].group('dataset', hist.Cat('process', 'Process'), pmap)

/uscms/home/jennetd/.local/lib/python3.6/site-packages/coffea/hist/hist_tools.py:346: RuntimeWarning: Not all requested indices present in <Cat (name=dataset) instance at 0x7f25f185b400>
  warnings.warn("Not all requested indices present in %r" % self, RuntimeWarning)


In [25]:
ptbins = [450, 500, 550, 600, 675, 800, 1200]

In [26]:
os.system('rm '+year+'/signalregion.root')
fout = uproot.create(year+'/signalregion.root')
for i,b in enumerate(ptbins[:-1]):
    for p in pmap.keys():  
        h = templates.integrate('region', 'signal').integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
        fout[p+"_pass_pt"+str(i+1)] = hist.export1d(h)
        h = templates.integrate('region', 'signal').integrate('pt1',int_range=slice(ptbins[i],ptbins[i+1])).integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
        fout[p+"_fail_pt"+str(i+1)] = hist.export1d(h)

fout.close()

KeyError: ()

In [ ]:
hist.plot1d(templates.integrate('region', 'signal').integrate('pt1',int_range=slice(ptbins[0],ptbins[1])).integrate('process','QCD'),overlay='ddb1',density=True)
plt.savefig(year+'/plot-all/qcd-msd-pt1.png')

In [ ]:
hist.plot1d(templates.integrate('region', 'signal').integrate('pt1',int_range=slice(ptbins[1],ptbins[2])).integrate('process','QCD'),overlay='ddb1',density=True)
plt.savefig(year+'/plot-all/qcd-msd-pt2.png')

In [ ]:
hist.plot1d(templates.integrate('region', 'signal').integrate('pt1',int_range=slice(ptbins[2],ptbins[3])).integrate('process','QCD'),overlay='ddb1',density=True)
plt.savefig(year+'/plot-all/qcd-msd-pt3.png')

In [ ]:
hist.plot1d(templates.integrate('region', 'signal').integrate('pt1',int_range=slice(ptbins[3],ptbins[4])).integrate('process','QCD'),overlay='ddb1',density=True)
plt.savefig(year+'/plot-all/qcd-msd-pt4.png')

In [ ]:
hist.plot1d(templates.integrate('region', 'signal').integrate('pt1',int_range=slice(ptbins[4],ptbins[5])).integrate('process','QCD'),overlay='ddb1',density=True)
plt.savefig(year+'/plot-all/qcd-msd-pt5.png')

In [ ]:
hist.plot1d(templates.integrate('region', 'signal').integrate('pt1',int_range=slice(ptbins[5],ptbins[6])).integrate('process','QCD'),overlay='ddb1',density=True)
plt.savefig(year+'/plot-all/qcd-msd-pt6.png')

In [ ]:
hist.plot1d(templates.integrate('region', 'signal').sum('pt1').integrate('process','QCD'),overlay='ddb1',density=True)
plt.savefig(year+'/plot-all/qcd-msd-all.png')

In [ ]:
os.system('rm '+year+'/muonCR.root')
fout = uproot.create(year+'/muonCR.root')
for p in pmap.keys():  
    h = templates.sum('pt1').integrate('region', 'muoncontrol').integrate('ddb1',int_range=slice(0.89,1)).integrate('process',p)
    fout[p+"_pass"] = hist.export1d(h)
    h = templates.sum('pt1').integrate('region', 'muoncontrol').integrate('ddb1',int_range=slice(0,0.89)).integrate('process',p)
    fout[p+"_fail"] = hist.export1d(h)

fout.close()

In [ ]:
templates.sum('pt1','msd1').integrate('region', 'signal').integrate('process','QCD').values()

In [ ]:
templates.sum('pt1','msd1').integrate('region', 'signal').integrate('process','singlet').values()